In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Cardiomegaly"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path= None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path= None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
264: 1950
1950: 1950
741: 1950
431: 1950
75: 1950
80: 1950
21: 1950
32: 1950
126: 1950
85: 1950
38: 1950
43: 1950
85: 1950
2: 1950
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
1947/1950 [============================>.] - ETA: 0s - loss: 4.8174 - leftLayer1_loss: 0.0895 - midLayer1_loss: 1.4631 - rightLayer1_loss: 0.9490 - leftLayer2_loss: 0.0747 - midLayer2_loss: 1.4067 - rightLayer2_loss: 0.8344
Epoch 00006: val_loss improved from 4.88240 to 4.79782, saving model to ./experiments/0.009_weights.h5
1950/1950 [==============================] - 5s 2ms/step - loss: 4.8177 - leftLayer1_loss: 0.0895 - midLayer1_loss: 1.4631 - rightLayer1_loss: 0.9490 - leftLayer2_loss: 0.0747 - midLayer2_loss: 1.4069 - rightLayer2_loss: 0.8345 - val_loss: 4.7978 - val_leftLayer1_loss: 0.0875 - val_midLayer1_loss: 1.4400 - val_rightLayer1_loss: 0.9102 - val_leftLayer2_loss: 0.0879 - val_midLayer2_loss: 1.3332 - val_rightLayer2_loss: 0.9390
Epoch 7/11
1942/1950 [============================>.] - ETA: 0s - loss: 4.7673 - leftLayer1_loss: 0.0857 - midLayer1_loss: 1.4636 - rightLayer1_loss: 0.9135 - leftLayer2_loss: 0.0691 - midLayer2_loss: 1.4144 - rightLayer2_loss: 0.8209


  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1950 steps, validate for 240 steps
Epoch 1/11
1946/1950 [============================>.] - ETA: 0s - loss: 6.4752 - leftLayer1_loss: 0.1195 - midLayer1_loss: 1.4282 - rightLayer1_loss: 1.7504 - leftLayer2_loss: 0.1234 - midLayer2_loss: 1.4943 - rightLayer2_loss: 1.5593
Epoch 00001: val_loss improved from inf to 6.00743, saving model to ./experiments/0.009999999999999998_weights.h5
1950/1950 [==============================] - 6s 3ms/step - loss: 6.4740 - leftLayer1_loss: 0.1195 - midLayer1_loss: 1.4282 - rightLayer1_loss: 1.7500 - leftLayer2_loss: 0.1234 - midLayer2_loss: 1.4941 - rightLayer2_loss: 1.5588 - val_loss: 6.0074 - val_leftLayer1_loss: 0.1166 - val_midLayer1_loss: 1.4038 - val_rightLayer1_loss: 1.5790 - val_leftLayer2_loss: 0.1151 - val_midLayer2_loss: 1.3495 - val_rightLayer2_loss: 1.4

1945/1950 [============================>.] - ETA: 0s - loss: 4.6870 - leftLayer1_loss: 0.0761 - midLayer1_loss: 1.4277 - rightLayer1_loss: 0.8399 - leftLayer2_loss: 0.0551 - midLayer2_loss: 1.4910 - rightLayer2_loss: 0.7971
Epoch 00011: val_loss improved from 4.60419 to 4.57581, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Cardiomegaly: 0.8574073846911628

auprc 0Cardiomegaly: 0.20694632452184125

auroc 1Cardiomegaly: 0.2628978817958114

auprc 1Cardiomegaly: 0.015852763179054947

auroc 2Cardiomegaly: 0.8162519711365999

auprc 2Cardiomegaly: 0.16494129919699602

auroc 3Cardiomegaly: 0.4941661276363928

auprc 3Cardiomegaly: 0.02353689193746181

auroc 4Cardiomegaly: 0.3793765444534454

auprc 4Cardiomegaly: 0.01882437149445452

auroc 5Cardiomegaly: 0.6593115179800212

auprc 5Cardiomegaly: 0.03967758656374086

mean auroc: 0.5782352379489056

mean

Epoch 4/11
1943/1950 [============================>.] - ETA: 0s - loss: 5.1499 - leftLayer1_loss: 0.1095 - midLayer1_loss: 1.4590 - rightLayer1_loss: 1.1128 - leftLayer2_loss: 0.0887 - midLayer2_loss: 1.4852 - rightLayer2_loss: 0.8946
Epoch 00004: val_loss improved from 5.27569 to 5.09116, saving model to ./experiments/0.010999999999999998_weights.h5
1950/1950 [==============================] - 5s 2ms/step - loss: 5.1505 - leftLayer1_loss: 0.1095 - midLayer1_loss: 1.4591 - rightLayer1_loss: 1.1128 - leftLayer2_loss: 0.0887 - midLayer2_loss: 1.4855 - rightLayer2_loss: 0.8948 - val_loss: 5.0912 - val_leftLayer1_loss: 0.1070 - val_midLayer1_loss: 1.4593 - val_rightLayer1_loss: 1.0424 - val_leftLayer2_loss: 0.0938 - val_midLayer2_loss: 1.3545 - val_rightLayer2_loss: 1.0341
Epoch 5/11
1935/1950 [============================>.] - ETA: 0s - loss: 5.0213 - leftLayer1_loss: 0.1047 - midLayer1_loss: 1.4584 - rightLayer1_loss: 1.0294 - leftLayer2_loss: 0.0804 - midLayer2_loss: 1.4884 - rightLayer

...

1935/1950 [============================>.] - ETA: 0s - loss: 4.5000 - leftLayer1_loss: 0.0812 - midLayer1_loss: 1.3773 - rightLayer1_loss: 0.8560 - leftLayer2_loss: 0.0533 - midLayer2_loss: 1.3401 - rightLayer2_loss: 0.7922
Epoch 00011: val_loss improved from 4.50714 to 4.47791, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Cardiomegaly: 0.7272147460440053

auprc 0Cardiomegaly: 0.07239700133387263

auroc 1Cardiomegaly: 0.5205780213098994

auprc 1Cardiomegaly: 0.0331741884504674

auroc 2Cardiomegaly: 0.7647029451733476

auprc 2Cardiomegaly: 0.14074092494743606

auroc 3Cardiomegaly: 0.6164934456906751

auprc 3Cardiomegaly: 0.03298490166825589

auroc 4Cardiomegaly: 0.3542490840417001

auprc 4Cardiomegaly: 0.0180365403277096

auroc 5Cardiomegaly: 0.757804143998694

auprc 5Cardiomegaly: 0.11584695463571833

mean auroc: 0.6235070643763869

mean aup

Epoch 4/11
1934/1950 [============================>.] - ETA: 0s - loss: 5.0145 - leftLayer1_loss: 0.1019 - midLayer1_loss: 1.2760 - rightLayer1_loss: 1.1171 - leftLayer2_loss: 0.0872 - midLayer2_loss: 1.5400 - rightLayer2_loss: 0.8923
Epoch 00004: val_loss improved from 5.09537 to 4.91306, saving model to ./experiments/0.012999999999999996_weights.h5
1950/1950 [==============================] - 5s 2ms/step - loss: 5.0161 - leftLayer1_loss: 0.1019 - midLayer1_loss: 1.2764 - rightLayer1_loss: 1.1171 - leftLayer2_loss: 0.0872 - midLayer2_loss: 1.5404 - rightLayer2_loss: 0.8931 - val_loss: 4.9131 - val_leftLayer1_loss: 0.0987 - val_midLayer1_loss: 1.2550 - val_rightLayer1_loss: 1.0523 - val_leftLayer2_loss: 0.0937 - val_midLayer2_loss: 1.3916 - val_rightLayer2_loss: 1.0219
Epoch 5/11
1941/1950 [============================>.] - ETA: 0s - loss: 4.8777 - leftLayer1_loss: 0.0976 - midLayer1_loss: 1.2764 - rightLayer1_loss: 1.0334 - leftLayer2_loss: 0.0795 - midLayer2_loss: 1.5310 - rightLayer

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1950 steps, validate for 240 steps
Epoch 1/11
1949/1950 [============================>.] - ETA: 0s - loss: 6.4730 - leftLayer1_loss: 0.1283 - midLayer1_loss: 1.4474 - rightLayer1_loss: 1.7198 - leftLayer2_loss: 0.1157 - midLayer2_loss: 1.5266 - rightLayer2_loss: 1.5353
Epoch 00001: val_loss improved from inf to 6.08039, saving model to ./experiments/0.013999999999999995_weights.h5
1950/1950 [==============================] - 6s 3ms/step - loss: 6.4729 - leftLayer1_loss: 0.1283 - midLayer1_loss: 1.4474 - rightLayer1_loss: 1.7197 - leftLayer2_loss: 0.1157 - midLayer2_loss: 1.5266 - rightLayer2_loss: 1.5352 - val_loss: 6.0804 - val_leftLayer1_loss: 0.1255 - val_midLayer1_loss: 1.4383 - val_rightLayer1_loss: 1.5814 - val_leftLayer2_loss: 0.1118 - val_midLayer2_loss: 1.4200 - val_rightLayer2_loss: 1.4

1944/1950 [============================>.] - ETA: 0s - loss: 4.7841 - leftLayer1_loss: 0.0873 - midLayer1_loss: 1.4472 - rightLayer1_loss: 0.8683 - leftLayer2_loss: 0.0534 - midLayer2_loss: 1.5340 - rightLayer2_loss: 0.7940
Epoch 00011: val_loss improved from 4.72367 to 4.69299, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Cardiomegaly: 0.6109862940839088

auprc 0Cardiomegaly: 0.03318195808001881

auroc 1Cardiomegaly: 0.48268670931414437

auprc 1Cardiomegaly: 0.023545074426601822

auroc 2Cardiomegaly: 0.7237734839881667

auprc 2Cardiomegaly: 0.06500090341746981

auroc 3Cardiomegaly: 0.5688486737968066

auprc 3Cardiomegaly: 0.031834447776353705

auroc 4Cardiomegaly: 0.3765890777604838

auprc 4Cardiomegaly: 0.019123454108296935

auroc 5Cardiomegaly: 0.5639859208909577

auprc 5Cardiomegaly: 0.02945283197040363

mean auroc: 0.5544783599724113

m

Epoch 4/11
1946/1950 [============================>.] - ETA: 0s - loss: 5.0180 - leftLayer1_loss: 0.1096 - midLayer1_loss: 1.3222 - rightLayer1_loss: 1.1726 - leftLayer2_loss: 0.0885 - midLayer2_loss: 1.4210 - rightLayer2_loss: 0.9042
Epoch 00004: val_loss improved from 5.18920 to 4.99408, saving model to ./experiments/0.014999999999999994_weights.h5
1950/1950 [==============================] - 5s 2ms/step - loss: 5.0181 - leftLayer1_loss: 0.1096 - midLayer1_loss: 1.3223 - rightLayer1_loss: 1.1725 - leftLayer2_loss: 0.0885 - midLayer2_loss: 1.4209 - rightLayer2_loss: 0.9043 - val_loss: 4.9941 - val_leftLayer1_loss: 0.1072 - val_midLayer1_loss: 1.3146 - val_rightLayer1_loss: 1.0992 - val_leftLayer2_loss: 0.0970 - val_midLayer2_loss: 1.3258 - val_rightLayer2_loss: 1.0502
Epoch 5/11
1943/1950 [============================>.] - ETA: 0s - loss: 4.8711 - leftLayer1_loss: 0.1054 - midLayer1_loss: 1.3217 - rightLayer1_loss: 1.0799 - leftLayer2_loss: 0.0808 - midLayer2_loss: 1.4190 - rightLayer

In [17]:
print(np.max(maxi))

0.8574073846911628
